### 1. Import libraries

In [1]:
import numpy as np
import pandas as pd
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
import json
from geopy.geocoders import Nominatim
import geocoder
import requests
from bs4 import BeautifulSoup
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium
print("Libraries imported.")

Libraries imported.


### 2. Web scrapping

In [2]:
# send the GET request
data = requests.get("https://en.wikipedia.org/wiki/Category:Neighbourhoods_in_Delhi").text

In [3]:
# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(data, 'html.parser')

In [4]:
# create a list to store neighborhood data
neighborhoodList = []

In [5]:
# append the data into the list
for row in soup.find_all("div", class_="mw-category")[0].findAll("li"):
    neighborhoodList.append(row.text)

In [6]:
# create a new DataFrame from the list
dl_df = pd.DataFrame({"Neighborhood": neighborhoodList})
dl_df = dl_df[1:]
dl_df.head()

,Neighborhood
1,Ashok Nagar (Delhi)
2,Ashok Vihar
3,Ashram Chowk
4,Babarpur
5,"Badarpur, Delhi"


In [7]:
# print the number of rows of the dataframe
dl_df.shape

(138, 1)

### 3. Get the geographical coordinates

In [8]:
# define a function to get coordinates
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Delhi, India'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [9]:
# call the function to get the coordinates, store in a new list using list comprehension
coords = [ get_latlng(neighborhood) for neighborhood in dl_df["Neighborhood"].tolist() ]

In [10]:
coords

[[28.692230000000052, 77.30124000000006],
 [28.69037000000003, 77.17609000000004],
 [28.710598435255907, 77.32696519316737],
 [28.50738000000007, 77.30346000000003],
 [28.50738000000007, 77.30346000000003],
 [28.65223022436032, 77.12941079026544],
 [28.800590000000057, 77.03473000000008],
 [28.549540000000036, 77.18167000000005],
 [28.699880000000064, 77.25906000000003],
 [28.595060000000046, 77.18573000000004],
 [28.656270000000063, 77.23232000000007],
 [28.67671000000007, 77.21767000000006],
 [28.633940000000052, 77.21968000000004],
 [28.60761000000008, 77.08714000000003],
 [28.654597885415757, 77.2333966005242],
 [28.62832000000003, 77.24727000000007],
 [28.60486000000003, 77.08511000000004],
 [28.560590000000047, 77.24678000000006],
 [28.57298000000003, 77.23357000000004],
 [28.591510000000028, 77.12945000000008],
 [28.699110000000076, 77.19105000000008],
 [28.592220036588714, 77.15998300657745],
 [28.684700000000078, 77.32774000000006],
 [28.679040000000043, 77.31476000000004],
 [

In [11]:
# create temporary dataframe to populate the coordinates into Latitude and Longitude
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

In [12]:
# merge the coordinates into the original dataframe
dl_df['Latitude'] = df_coords['Latitude']
dl_df['Longitude'] = df_coords['Longitude']

In [13]:
# check the neighborhoods and the coordinates
print(dl_df.shape)
dl_df

(138, 3)


,Neighborhood,Latitude,Longitude
1,Ashok Nagar (Delhi),28.690370,77.176090
2,Ashok Vihar,28.710598,77.326965
3,Ashram Chowk,28.507380,77.303460
4,Babarpur,28.507380,77.303460
5,"Badarpur, Delhi",28.652230,77.129411
6,Bali Nagar,28.800590,77.034730
7,Bawana,28.549540,77.181670
8,Ber Sarai,28.699880,77.259060
9,Bhajanpura,28.595060,77.185730
10,Chanakyapuri,28.656270,77.232320


In [14]:
# save the DataFrame as CSV file
dl_df.to_csv("dl_df.csv", index=False)

### 4. Create a map of Kuala Lumpur with neighborhoods superimposed on top

In [15]:
address = 'Delhi, India'

geolocator = Nominatim(user_agent="Coursera-Capstone")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Delhi, India {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Delhi, India 28.6517178, 77.2219388.


In [16]:
dl_df.dropna(axis = 0, inplace = True)

In [17]:
# create map of Toronto using latitude and longitude values
map_dl = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(dl_df['Latitude'], dl_df['Longitude'], dl_df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_dl)  
    
map_dl

In [18]:
# save the map as HTML file
map_dl.save('map_dl.html')

### 5. Use the Foursquare API to explore the neighborhoods

In [1]:
# define Foursquare Credentials and Version
CLIENT_ID = 'your Foursquare ID' # your Foursquare ID
CLIENT_SECRET = 'your Foursquare Secret' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: your Foursquare ID
CLIENT_SECRET:your Foursquare Secret


#### Now, let's get the top 100 venues that are within a radius of 3000 meters.

In [20]:
radius = 3000
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(dl_df['Latitude'], dl_df['Longitude'], dl_df['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [21]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(8241, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Ashok Nagar (Delhi),28.69037,77.17609,Major Dhyan Chand Sports Complex,28.684029,77.167487,Athletics & Sports
1,Ashok Nagar (Delhi),28.69037,77.17609,Big Yellow Door,28.693245,77.204948,Café
2,Ashok Nagar (Delhi),28.69037,77.17609,Bellagio,28.696361,77.180021,Asian Restaurant
3,Ashok Nagar (Delhi),28.69037,77.17609,Ever Bake,28.706419,77.162607,Bakery
4,Ashok Nagar (Delhi),28.69037,77.17609,Shakti Nagar,28.682103,77.196706,Playground


#### Let's check how many venues were returned for each neighorhood

In [22]:
venues_df.groupby(["Neighborhood"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
Ashok Nagar (Delhi),53,53,53,53,53,53
Ashok Vihar,5,5,5,5,5,5
Ashram Chowk,9,9,9,9,9,9
Babarpur,9,9,9,9,9,9
"Badarpur, Delhi",86,86,86,86,86,86
Bali Nagar,1,1,1,1,1,1
Bawana,100,100,100,100,100,100
Ber Sarai,8,8,8,8,8,8
Bhajanpura,93,93,93,93,93,93


#### Let's find out how many unique categories can be curated from all the returned venues

In [23]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 205 uniques categories.


In [24]:
# print out any 50 categories from the list of categories
venues_df['VenueCategory'].unique()[:50]

array(['Athletics & Sports', 'Café', 'Asian Restaurant', 'Bakery',
       'Playground', 'Multiplex', 'Snack Place', 'Fast Food Restaurant',
       'Indian Restaurant', 'Sandwich Place', 'Breakfast Spot',
       'Donut Shop', 'Chinese Restaurant', 'Pizza Place',
       'Miscellaneous Shop', 'Diner', 'Coffee Shop', 'Department Store',
       'South Indian Restaurant', 'Shopping Mall', 'Food Truck',
       'Arts & Crafts Store', 'ATM', 'Print Shop', 'Plaza', 'Smoke Shop',
       'Indian Sweet Shop', 'Hotel', 'Restaurant', 'Punjabi Restaurant',
       'Historic Site', 'American Restaurant', 'Market', 'BBQ Joint',
       'Clothing Store', 'Hookah Bar', 'Vegetarian / Vegan Restaurant',
       'Sculpture Garden', 'Sports Bar', 'Pub', 'Garden Center',
       'Ice Cream Shop', 'Italian Restaurant', 'Furniture / Home Store',
       'Food Court', 'Fried Chicken Joint', 'Arcade', 'Bar',
       'Light Rail Station', 'Gym'], dtype=object)

In [25]:
# check if the results contain "Restaurant"
"Restaurant" in venues_df['VenueCategory'].unique()

True

### 6. Analyze Each Neighborhood

In [26]:
# one hot encoding
dl_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
dl_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [dl_onehot.columns[-1]] + list(dl_onehot.columns[:-1])
dl_onehot = dl_onehot[fixed_columns]

print(dl_onehot.shape)
dl_onehot.head()

(8241, 206)


,Neighborhoods,ATM,Airport,Airport Food Court,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,BBQ Joint,Bakery,Bar,Bed & Breakfast,Beer Garden,Bengali Restaurant,Big Box Store,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Breakfast Spot,Brewery,Burger Joint,Burmese Restaurant,Burrito Place,Bus Station,Business Service,Café,Candy Store,Cantonese Restaurant,Castle,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Cricket Ground,Deli / Bodega,Department Store,Dessert Shop,Diner,Donut Shop,Dumpling Restaurant,Electronics Store,English Restaurant,Event Space,Fabric Shop,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Golf Course,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hardware Store,Health & Beauty Service,Hindu Temple,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hookah Bar,Hostel,Hotel,Hotel Bar,Hotel Pool,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indian Sweet Shop,Indie Movie Theater,Irani Cafe,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Karaoke Bar,Karnataka Restaurant,Korean Restaurant,Light Rail Station,Lounge,Luggage Store,Market,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Mosque,Motel,Motorcycle Shop,Movie Theater,Moving Target,Mughlai Restaurant,Multicuisine Indian Restaurant,Multiplex,Museum,Music Venue,Neighborhood,Nightclub,Nightlife Spot,North Indian Restaurant,Northeast Indian Restaurant,Other Great Outdoors,Other Nightlife,Palace,Paper / Office Supplies Store,Park,Parsi Restaurant,Performing Arts Venue,Pizza Place,Planetarium,Platform,Playground,Plaza,Pool,Portuguese Restaurant,Print Shop,Pub,Public Art,Punjabi Restaurant,Racetrack,Record Shop,Recreation Center,Restaurant,River,Road,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Sculpture Garden,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Skating Rink,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,South Indian Restaurant,Spa,Speakeasy,Spiritual Center,Sporting Goods Shop,Sports Bar,Stadium,Sushi Restaurant,Tapas Restaurant,Tea Room,Temple,Tex-Mex Restaurant,Thai Restaurant,Theater,Theme Park,Tibetan Restaurant,Tourist Information Center,Toy / Game Store,Track,Trail,Train Station,Turkish Restaurant,University,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio,Zoo
0,Ashok Nagar (Delhi),0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Ashok Nagar (Delhi),0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Ashok Nagar (Delhi),0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [27]:
dl_grouped = dl_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(dl_grouped.shape)
dl_grouped

(137, 206)


,Neighborhoods,ATM,Airport,Airport Food Court,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,BBQ Joint,Bakery,Bar,Bed & Breakfast,Beer Garden,Bengali Restaurant,Big Box Store,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Breakfast Spot,Brewery,Burger Joint,Burmese Restaurant,Burrito Place,Bus Station,Business Service,Café,Candy Store,Cantonese Restaurant,Castle,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Cricket Ground,Deli / Bodega,Department Store,Dessert Shop,Diner,Donut Shop,Dumpling Restaurant,Electronics Store,English Restaurant,Event Space,Fabric Shop,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Golf Course,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hardware Store,Health & Beauty Service,Hindu Temple,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hookah Bar,Hostel,Hotel,Hotel Bar,Hotel Pool,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indian Sweet Shop,Indie Movie Theater,Irani Cafe,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Karaoke Bar,Karnataka Restaurant,Korean Restaurant,Light Rail Station,Lounge,Luggage Store,Market,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Mosque,Motel,Motorcycle Shop,Movie Theater,Moving Target,Mughlai Restaurant,Multicuisine Indian Restaurant,Multiplex,Museum,Music Venue,Neighborhood,Nightclub,Nightlife Spot,North Indian Restaurant,Northeast Indian Restaurant,Other Great Outdoors,Other Nightlife,Palace,Paper / Office Supplies Store,Park,Parsi Restaurant,Performing Arts Venue,Pizza Place,Planetarium,Platform,Playground,Plaza,Pool,Portuguese Restaurant,Print Shop,Pub,Public Art,Punjabi Restaurant,Racetrack,Record Shop,Recreation Center,Restaurant,River,Road,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Sculpture Garden,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Skating Rink,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,South Indian Restaurant,Spa,Speakeasy,Spiritual Center,Sporting Goods Shop,Sports Bar,Stadium,Sushi Restaurant,Tapas Restaurant,Tea Room,Temple,Tex-Mex Restaurant,Thai Restaurant,Theater,Theme Park,Tibetan Restaurant,Tourist Information Center,Toy / Game Store,Track,Trail,Train Station,Turkish Restaurant,University,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio,Zoo
0,Ashok Nagar (Delhi),0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.018868,0.018868,0.018868,0.00,0.000000,0.037736,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.037736,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.037736,0.000000,0.00,0.000000,0.018868,0.000000,0.000000,0.150943,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.018868,0.000000,0.018868,0.037736,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.094340,0.000000,0.00,0.000000,0.000000,0.000000,0.018868,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.113208,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.018868,0.000000,0.000000,0.000000,0.000000,0.000000

In [28]:
len(dl_grouped[dl_grouped["Restaurant"] > 0])

89

#### Create a new DataFrame for Shopping Mall data only

In [30]:
dl_resturant = dl_grouped[["Neighborhoods","Restaurant"]]
dl_resturant.head()

,Neighborhoods,Restaurant
0,Ashok Nagar (Delhi),0.000000
1,Ashok Vihar,0.000000
2,Ashram Chowk,0.111111
3,Babarpur,0.111111
4,"Badarpur, Delhi",0.034884


### 7. Cluster Neighborhoods
Run k-means to cluster the neighborhoods in Delhi into 5 clusters.

In [31]:
# set number of clusters
dclusters = 5

dl_clustering = dl_resturant.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=dclusters, random_state=0).fit(dl_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([3, 3, 1, 1, 2, 3, 0, 3, 0, 2])

In [32]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
dl_merged = dl_resturant.copy()

# add clustering labels
dl_merged["Cluster Labels"] = kmeans.labels_

In [33]:
dl_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
dl_merged.head()

,Neighborhood,Restaurant,Cluster Labels
0,Ashok Nagar (Delhi),0.000000,3
1,Ashok Vihar,0.000000,3
2,Ashram Chowk,0.111111,1
3,Babarpur,0.111111,1
4,"Badarpur, Delhi",0.034884,2


In [34]:
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
dl_merged = dl_merged.join(dl_df.set_index("Neighborhood"), on="Neighborhood")

print(dl_merged.shape)
dl_merged.head() # check the last columns!

(137, 5)


,Neighborhood,Restaurant,Cluster Labels,Latitude,Longitude
0,Ashok Nagar (Delhi),0.000000,3,28.690370,77.176090
1,Ashok Vihar,0.000000,3,28.710598,77.326965
2,Ashram Chowk,0.111111,1,28.507380,77.303460
3,Babarpur,0.111111,1,28.507380,77.303460
4,"Badarpur, Delhi",0.034884,2,28.652230,77.129411


In [36]:
# sort the results by Cluster Labels
print(dl_merged.shape)
dl_merged.sort_values(["Cluster Labels"], inplace=True)
dl_merged

(137, 5)


,Neighborhood,Restaurant,Cluster Labels,Latitude,Longitude
88,Okhla,0.020000,0,28.654340,77.232580
25,East Patel Nagar,0.016949,0,28.521130,77.180170
26,"Gadaipur, Mehrauli, New Delhi",0.020000,0,28.528450,77.203540
123,Shiv Ram Park,0.021505,0,28.652650,77.121390
100,Rajendra Place,0.015625,0,28.645620,77.122090
39,Jangpura,0.020000,0,28.546380,77.188970
115,Seemapuri subdivision,0.023810,0,28.653900,77.296410
112,Sarita Vihar,0.020000,0,28.575600,77.193640
48,"Khanpur, Delhi",0.020000,0,28.657260,77.222840
24,"Dwarka, Delhi",0.020833,0,28.648170,77.178330


#### Finally, let's visualize the resulting clusters

In [38]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(dclusters)
ys = [i+x+(i*x)**2 for i in range(dclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(dl_merged['Latitude'], dl_merged['Longitude'], dl_merged['Neighborhood'], dl_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [39]:
# save the map as HTML file
map_clusters.save('map_clusters.html')

### 8. Examine Clusters

#### Cluster 0

In [40]:
dl_merged.loc[dl_merged['Cluster Labels'] == 0]

,Neighborhood,Restaurant,Cluster Labels,Latitude,Longitude
88,Okhla,0.020000,0,28.654340,77.232580
25,East Patel Nagar,0.016949,0,28.521130,77.180170
26,"Gadaipur, Mehrauli, New Delhi",0.020000,0,28.528450,77.203540
123,Shiv Ram Park,0.021505,0,28.652650,77.121390
100,Rajendra Place,0.015625,0,28.645620,77.122090
39,Jangpura,0.020000,0,28.546380,77.188970
115,Seemapuri subdivision,0.023810,0,28.653900,77.296410
112,Sarita Vihar,0.020000,0,28.575600,77.193640
48,"Khanpur, Delhi",0.020000,0,28.657260,77.222840
24,"Dwarka, Delhi",0.020833,0,28.648170,77.178330


#### Cluster 1

In [41]:
dl_merged.loc[dl_merged['Cluster Labels'] == 1]

,Neighborhood,Restaurant,Cluster Labels,Latitude,Longitude
2,Ashram Chowk,0.111111,1,28.50738,77.30346
3,Babarpur,0.111111,1,28.50738,77.30346
65,Mayur Vihar,0.086957,1,28.61125,77.33406


#### Cluster 2

In [42]:
dl_merged.loc[dl_merged['Cluster Labels'] == 2]

,Neighborhood,Restaurant,Cluster Labels,Latitude,Longitude
32,Greater Kailash,0.030000,2,28.55897,77.204620
106,"Sadar Bazaar, Delhi",0.030000,2,28.56583,77.199070
57,Lodhi Colony,0.035294,2,28.60148,77.229010
59,Madanpur Khadar JJ Colony,0.029851,2,28.57223,77.263570
131,Vasant Kunj,0.030000,2,28.56494,77.161310
97,Punjabi Bagh,0.040000,2,28.61840,77.215481
98,Raisina Hill,0.030000,2,28.59075,77.227490
31,Govindpuri,0.040000,2,28.54849,77.236670
101,Rajouri Garden,0.030000,2,28.56553,77.177190
4,"Badarpur, Delhi",0.034884,2,28.65223,77.129411


#### Cluster 3

In [43]:
dl_merged.loc[dl_merged['Cluster Labels'] == 3]

,Neighborhood,Restaurant,Cluster Labels,Latitude,Longitude
93,Pandav Nagar,0.0,3,28.669330,77.091730
96,Pitam Pura,0.0,3,28.666340,77.125000
95,Patel Nagar,0.0,3,28.695900,77.137250
92,Pamposh Enclave,0.0,3,28.614580,77.275740
0,Ashok Nagar (Delhi),0.0,3,28.690370,77.176090
103,"Rani Bagh, Delhi",0.0,3,28.733560,77.104010
104,"Rohini, Delhi",0.0,3,28.683720,77.197470
105,Roop Nagar,0.0,3,28.590280,77.120140
107,Safdarjung (Delhi),0.0,3,28.605880,77.095520
111,Sarai Kale Khan,0.0,3,28.550380,77.283410


#### Cluster 4

In [44]:
dl_merged.loc[dl_merged['Cluster Labels'] == 4]

,Neighborhood,Restaurant,Cluster Labels,Latitude,Longitude
127,Sriniwaspuri,0.050847,4,28.640890,77.089220
17,Dayanand Colony,0.050000,4,28.572980,77.233570
51,Kirti Nagar,0.050000,4,28.574350,77.224190
86,Nizamuddin East,0.060000,4,28.590820,77.244340
91,Palika Bazaar,0.060000,4,28.546774,77.244757
132,"Vasant Vihar, Delhi",0.057143,4,28.600150,77.316630
85,Nigambodh Ghat,0.047619,4,28.601240,77.264521
29,Gole Market,0.060000,4,28.603040,77.232690
53,"Krishna Nagar, Delhi",0.050000,4,28.570260,77.247000
30,"Golf Links, New Delhi",0.075949,4,28.535080,77.265120


#### Observation

There are so many restaurants, which are located in different areas of the central part of Delhi. If we observe the concentration of restaurant based on cluster then we can say `cluster 1> cluster 4> cluster 2> cluster 0> cluster 3`. From here we can easily understand that the competition is very high in cluster 1 and very less to no competition in cluster 0. That means this is great opportunity to open a restaurant in cluster 3. Therefore, **this project recommends property developers to capitalize on these findings to open new restaurant in neighbourhoods in cluster 3 with little to no competition**. Property developers with unique selling propositions to stand out from the competition can also open new restaurant in neighbourhoods in cluster 0,2 and 4 with moderate competition. Lastly, property developers are advised to avoid neighbourhoods in cluster 2 which already have high concentration of restaurant and suffering from intense competition. 